In [1]:
import cv2
import numpy as np

# taking image path or name from user
path = input('Enter Image path to recognize shape: ')
# Read image
img = cv2.imread(path)

# check size (bounding box) is square
def isSquare(siz):
    ratio = abs(siz[0] - siz[1]) / siz[0]
    #print (siz, ratio)
    if ratio < 0.1:
        return True
    else:
        return False

# chekc circle from the arc length ratio
def isCircle(cnt):
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    len = cv2.arcLength(cnt, True)
    ratio = abs(len - np.pi * 2.0 * radius) / (np.pi * 2.0 * radius)
    #print(ratio)
    if ratio < 0.1:
        return True
    else:
        return False

# Convert to gray-scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Binary
thr,dst = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

# clean up
kd = np.ones((3,3), dtype=np.uint8)
ke = np.ones((5,5), dtype=np.uint8)

dst = cv2.morphologyEx(dst,cv2.MORPH_CLOSE,ke)
for i in range(1):
     dst = cv2.dilate(dst, kd)

# find contours with hierachy
cont, hier = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# each contoure
for i in range(len(cont)):
    c = cont[i]
    h = hier[0,i]
    
    if h[2] == -1 and h[3] == 0:
        # no child and parent is image outer
        img = cv2.drawContours(img, cont, i, (0,0,255),-1)
    elif h[3] == 0 and hier[0,h[2]][2] == -1:
        # with child
        if isCircle(c):
            if isCircle(cont[h[2]]):
                # double circle
                img = cv2.drawContours(img, cont, i, (0,255,0),-1)
        else:
            # 1 child and shape bounding box is not squre 
            if not isSquare(cv2.minAreaRect(c)[1]) and hier[0,h[2]][0] == -1 and hier[0,h[2]][1] == -1:
                img = cv2.drawContours(img, cont, i, (255,0, 0),-1)
            else:
                img = cv2.drawContours(img, cont, i, (0,255,255),-1)
    elif h[0] == -1 and not isCircle(c) and hier[0,h[2]][0] == -1:
        img = cv2.drawContours(img, cont, i, (255,0,127),-1)
        
# scaling the image
scale_percent = 40 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
cv2.imshow("Image", img)
cv2.imwrite("all-parts-output.png", img)
cv2.waitKey()
cv2.destroyAllWindows()

Enter Image path to recognize shape: all-parts.png
